In [1]:
import my_utils
import pandas as pd

In [2]:
my_utils.makeDirectories(verbose=True)

Created or already existed: data/complete_dfs
Created or already existed: data/pjm
Created or already existed: data/weather
Created or already existed: models


In [4]:
my_utils.checkInconsistencies()

2016 did not match! The difference is: {'AECO', 'VMEU', 'OVEC', 'AE'}
2017 did not match! The difference is: {'OVEC'}
Finished checking inconsistencies!


In [3]:
my_utils.getAllWeatherDfs(verbose=True)
aeco_weather_df = my_utils.getWeatherDf("AECO", verbose=True)
print(aeco_weather_df.head())

Cached weather file for AECO was found
Cached weather file for AEPAPT was found
Cached weather file for AEPIMP was found
Cached weather file for AEPKPT was found
Cached weather file for AEPOPT was found
Cached weather file for AP was found
Cached weather file for BC was found
Cached weather file for CE was found
Cached weather file for DAY was found
Cached weather file for DEOK was found
Cached weather file for DOM was found
Added new cached weather file for DPLCO using zip code:19901
Cached weather file for DUQ was found
Cached weather file for EASTON was found
Cached weather file for EKPC was found
Cached weather file for JC was found
Cached weather file for ME was found
Cached weather file for OE was found
Added new cached weather file for OVEC using zip code:45661
Cached weather file for PAPWR was found
Cached weather file for PE was found
Cached weather file for PEPCO was found
Cached weather file for PLCO was found
Cached weather file for PN was found
Cached weather file for PS w

In [5]:
print(my_utils.getUSHolidays())
#print(my_utils.getUSHolidays().keys())

{datetime.date(2018, 1, 1): "New Year's Day", datetime.date(2018, 5, 28): 'Memorial Day', datetime.date(2018, 7, 4): 'Independence Day', datetime.date(2018, 9, 3): 'Labor Day', datetime.date(2018, 11, 11): 'Veterans Day', datetime.date(2018, 11, 12): 'Veterans Day (observed)', datetime.date(2018, 11, 22): 'Thanksgiving Day', datetime.date(2018, 12, 25): 'Christmas Day', datetime.date(2018, 1, 15): 'Martin Luther King Jr. Day', datetime.date(2018, 2, 19): "Washington's Birthday", datetime.date(2018, 10, 8): 'Columbus Day', datetime.date(2019, 1, 1): "New Year's Day", datetime.date(2019, 5, 27): 'Memorial Day', datetime.date(2019, 7, 4): 'Independence Day', datetime.date(2019, 9, 2): 'Labor Day', datetime.date(2019, 11, 11): 'Veterans Day', datetime.date(2019, 11, 28): 'Thanksgiving Day', datetime.date(2019, 12, 25): 'Christmas Day', datetime.date(2019, 1, 21): 'Martin Luther King Jr. Day', datetime.date(2019, 2, 18): "Washington's Birthday", datetime.date(2019, 10, 14): 'Columbus Day', 

In [12]:
# Thanksgiving 2024 (Nov 28, 2024)
#test_dt = datetime.datetime(2024, 11, 29, 15, 0)  # 3pm on Thanksgiving Day
#print(my_utils.isHoliday(test_dt, getUSHolidays()))
#print(my_utils.isBlackFriday(test_dt, getUSHolidays()))

In [6]:
aeco_energy_df = my_utils.getEnergyDf("AECO")
print(aeco_energy_df.tail())

Cached pjm fresh file was found
Loading fresh data in addition to historical data
                          datetime_beginning_ept nerc_region mkt_region zone  \
datetime_beginning_utc                                                         
2025-11-10 00:00:00+00:00   11/9/2025 7:00:00 PM         RFC     MIDATL   AE   
2025-11-10 01:00:00+00:00   11/9/2025 8:00:00 PM         RFC     MIDATL   AE   
2025-11-10 02:00:00+00:00   11/9/2025 9:00:00 PM         RFC     MIDATL   AE   
2025-11-10 03:00:00+00:00  11/9/2025 10:00:00 PM         RFC     MIDATL   AE   
2025-11-10 04:00:00+00:00  11/9/2025 11:00:00 PM         RFC     MIDATL   AE   

                          load_area       mw  is_verified  
datetime_beginning_utc                                     
2025-11-10 00:00:00+00:00      AECO  980.590        False  
2025-11-10 01:00:00+00:00      AECO  952.221        False  
2025-11-10 02:00:00+00:00      AECO  908.209        False  
2025-11-10 03:00:00+00:00      AECO  855.014        False

In [18]:
#df_augmented = add_features(aeco_energy_df, aeco_weather_df, False)
#print(df_augmented)

In [4]:
# print(getCompleteDf("AECO").head())
load_area_to_complete_df = my_utils.getAllCompleteDfs()

In [5]:
aeco_prediction_features = my_utils.getPredictionFeatures("AECO")
print(aeco_prediction_features)

                          datetime_beginning_ept nerc_region mkt_region zone  \
2025-11-17 05:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 06:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 07:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 08:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 09:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 10:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 11:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 12:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 13:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 14:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 15:00:00+00:00                    NaN         NaN        NaN  NaN   
2025-11-17 16:00:00+00:00               